In [ ]:
with open('tokenized_datasets1.pkl', 'rb') as f:
    loaded_datasets = pickle.load(f)

In [ ]:
train_dataset = loaded_datasets['train']
test_dataset = loaded_datasets['eval']

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15189
})

In [ ]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3798
})

In [ ]:
from transformers import GPT2LMHeadModel, Trainer


model = GPT2LMHeadModel.from_pretrained('gpt2')


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,  # Log every 50 steps
    evaluation_strategy="epoch",
    fp16=True,  # Enable mixed precision training
    save_total_limit=2,  # Limit the total amount of checkpoints. Delete the older checkpoints.
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=loaded_datasets['train'],
    eval_dataset=loaded_datasets['eval'],
)


trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,4.627700,4.506895
2,4.458800,4.425511
3,4.372300,4.404546


TrainOutput(global_step=5697, training_loss=4.569023862185553, metrics={'train_runtime': 3084.681, 'train_samples_per_second': 14.772, 'train_steps_per_second': 1.847, 'total_flos': 1.1906294022144e+16, 'train_loss': 4.569023862185553, 'epoch': 3.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 4.40454626083374, 'eval_runtime': 76.2752, 'eval_samples_per_second': 49.793, 'eval_steps_per_second': 6.227, 'epoch': 3.0}


In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
model.save_pretrained('./nano_gpt_model')
tokenizer.save_pretrained('./nano_gpt_model')

('./nano_gpt_model/tokenizer_config.json',
 './nano_gpt_model/special_tokens_map.json',
 './nano_gpt_model/vocab.json',
 './nano_gpt_model/merges.txt',
 './nano_gpt_model/added_tokens.json')

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import torch

input_text = "generaete a short 10 stroy ,Once upon a time in a faraway land, there lived a brave knight."


inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)


with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=100,
        num_return_sequences=1,
        do_sample=True,  # Set do_sample to True
        temperature=0.7,  # Increase temperature for more randomness
        top_k=50,  # Use top-k sampling
        top_p=0.9,  # Use top-p sampling
        repetition_penalty=1.2  # Penalize repetitive tokens
    )


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


generaete a short 10 stroy ,Once upon a time in a faraway land, there lived a brave knight. And with that noble name and glory he was called "The Great Knight." He had been raised from the dead by his father's death to be Lord Commander of Thessalonica (the realm) within its borders; but when it fell into ruin at sea as It has already done this year for so many years now is never more known than here today . Nowhere else


In [ ]:
def generate_text(model, tokenizer, input_text, max_length=100, num_return_sequences=1, temperature=0.7, top_k=50, top_p=0.9, repetition_penalty=1.2):
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Define some input prompts
input_prompts = [
    "Once upon a time in a faraway land, there lived a brave knight.",
    "In a world where technology and magic coexist, a young inventor discovers a hidden power.",
    "The year is 2050, and humanity has colonized Mars. The first Martian-born child is about to be born.",
    "In the bustling city of New York, a detective stumbles upon a mysterious artifact that changes his life forever.",
    "A group of friends embark on a journey to find a hidden treasure, but they soon realize that they are not alone."
]

# Generate text for each input prompt
for prompt in input_prompts:
    print(f"Input Prompt: {prompt}")
    generated_text = generate_text(model, tokenizer, prompt)
    print(f"Generated Text: {generated_text}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input Prompt: Once upon a time in a faraway land, there lived a brave knight.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: Once upon a time in a faraway land, there lived a brave knight. He was an old man with gold-rimmed glasses and white gloves of silver armor; but his armour wore out all manner of good worksmanship to the point where he could not even keep up at once when one fell into battle."
This tale is told by Robert Rauch: "There were three sons who fought on this side for more than twenty years before their father's death—and they are named after

Input Prompt: In a world where technology and magic coexist, a young inventor discovers a hidden power.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: In a world where technology and magic coexist, a young inventor discovers a hidden power.
This is the story of an extraordinary genius who finds himself in danger when his own life depends on it - even if he can't get out alive!

Input Prompt: The year is 2050, and humanity has colonized Mars. The first Martian-born child is about to be born.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: The year is 2050, and humanity has colonized Mars. The first Martian-born child is about to be born.
I was lucky enough in the summer of 2012 to attend an event at NASA's Jet Propulsion Laboratory (JPL) that brought together many people who were interested but not necessarily passionate for exploring this unexplored planet—a day I would have never attended without being there with my parents when we discovered Curiosity on August 4th 2013! This little girl just got off a flight from

Input Prompt: In the bustling city of New York, a detective stumbles upon a mysterious artifact that changes his life forever.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: In the bustling city of New York, a detective stumbles upon a mysterious artifact that changes his life forever.
WWE: The Ultimate Matchmaker! WWE 2K18 - Review & Ratings Wrestling (Slammiversary) DVD / Normal Retail | Rental 01/11 6" x 9-inches $9 US 12 32 https://www4wdtwneats.com/?p=36298815&id=47163389 1 http [cite]http:/sn

Input Prompt: A group of friends embark on a journey to find a hidden treasure, but they soon realize that they are not alone.
Generated Text: A group of friends embark on a journey to find a hidden treasure, but they soon realize that they are not alone. Written by Anonymous



In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from nltk.translate.meteor_score import meteor_score

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def generate_text(model, tokenizer, input_text, max_length=100, num_return_sequences=1, temperature=0.7, top_k=50, top_p=0.9, repetition_penalty=1.2):
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Example usage
input_prompts = [
    "Once upon a time in a faraway land, there lived a brave knight.",
    "In a world where technology and magic coexist, a young inventor discovers a hidden power.",
    "The year is 2050, and humanity has colonized Mars. The first Martian-born child is about to be born.",
    "In the bustling city of New York, a detective stumbles upon a mysterious artifact that changes his life forever.",
    "A group of friends embark on a journey to find a hidden treasure, but they soon realize that they are not alone."
]

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

for prompt in input_prompts:
    print(f"Input Prompt: {prompt}")
    generated_text = generate_text(model, tokenizer, prompt)
    print(f"Generated Text: {generated_text}\n")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input Prompt: Once upon a time in a faraway land, there lived a brave knight.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: Once upon a time in a faraway land, there lived a brave knight. He was an old man of the city and he had taken refuge under his own roof from all that happened to him on earth; but when this fell apart at last into many other ways it seemed as if something strange were happening."
The young lord remembered only one thing about these days: "In those few months I never thought we would be able for any length or strength against them again until they attacked us with their

Input Prompt: In a world where technology and magic coexist, a young inventor discovers a hidden power.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: In a world where technology and magic coexist, a young inventor discovers a hidden power. The man who invented it is now the first to use this ability in his own life!


Input Prompt: The year is 2050, and humanity has colonized Mars. The first Martian-born child is about to be born.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: The year is 2050, and humanity has colonized Mars. The first Martian-born child is about to be born. It's the second generation of humans on Earth since ancient times (before we started living in space).
A few months ago I wrote a post that said "the story behind all this isn't so much what you can do or say as it does how long your life will take." Nowadays people have become more aware than ever before where they live for their entire lives — from

Input Prompt: In the bustling city of New York, a detective stumbles upon a mysterious artifact that changes his life forever.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: In the bustling city of New York, a detective stumbles upon a mysterious artifact that changes his life forever.
The most famous story in American history is about this boy's quest to find an ancient tomb for himself and save humanity from one hellish disease known as "Curse-Killer." It involves all manner ornaments including some very interesting facts such: 1) The child who discovers it had never heard before was possessed by supernatural forces; 2] His father said he'd found another

Input Prompt: A group of friends embark on a journey to find a hidden treasure, but they soon realize that they are not alone.
Generated Text: A group of friends embark on a journey to find a hidden treasure, but they soon realize that they are not alone. The team must help each other and uncover the truth behind what is happening inside their own head—but it's up to them to save humanity from an unknown threat!
 (Source: AnimeHelpers)




In [ ]:
df = pd.read_parquet('processed_dataset.parquet')

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu_score(generated_text, reference_texts):
    generated_tokens = word_tokenize(generated_text)
    reference_tokens = [word_tokenize(reference_text) for reference_text in reference_texts]
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing_function)

# Example usage
reference_texts = df['text'].tolist()  # Use the entire dataset as reference texts
generated_text = generate_text(model, tokenizer, input_prompts[0])
bleu_score = calculate_bleu_score(generated_text, reference_texts)
print(f"BLEU Score: {bleu_score:.4f}")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BLEU Score: 0.0195


In [ ]:
def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
        perplexity = torch.exp(loss).item()
    return perplexity

# Example usage
perplexity = calculate_perplexity(model, tokenizer, generated_text)
print(f"Perplexity: {perplexity:.4f}")


Perplexity: 22.0721
